In [ ]:
%matplotlib inline

from fastai.vision import * #TODO

In [ ]:
classes = ["gothic","ancient_greek","tudor","traditional_japanese","modernist"]

path = Path("data/arch")
dirs = [path/x for x in classes]

for d in dirs:
    d.mkdir(parents=True,exist_ok=True)

path.ls()

In [ ]:
def remove_blank_lines(s):
    with open(s,"r") as f:
        q = f.read()
        q = q.split("\n")
        d = "".join([f"{x}\n" for x in q if x])
    with open(s,"w") as f:
        f.write(d)

for c in classes:
    remove_blank_lines(path/f"{c}.txt")

In [ ]:
def download():
    for c,d in zip(classes,dirs):
        download_images(path/f"{c}.txt",d,max_pics=500)
#download()

bs = 32

In [ ]:
for d in dirs:
    print(d)
    verify_images(d,delete=True,max_size=512)

In [ ]:
np.random.seed(42)
data = ImageDataBunch.from_folder(path,train=".",
    valid_pct=0.2,ds_tfms=get_transforms(),bs=bs,
    size=224,num_workers=4).normalize(imagenet_stats)

In [ ]:
data.classes, data.c, len(data.train_ds), len(data.valid_ds)

In [ ]:
data.show_batch(rows=3,figsize=(7,8))

In [ ]:
learn = cnn_learner(data,models.resnet34,metrics=error_rate)

In [ ]:
learn.fit_one_cycle(6)

In [ ]:
learn.save("stage-1")

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(2,max_lr=slice(1e-5,1e-4))

In [ ]:
learn.save("stage-2")

In [ ]:
learn.load("stage-2")

interp = ClassificationInterpretation.from_learner(learn)

interp.plot_confusion_matrix()

In [ ]:
from fastai.widgets import *

In [ ]:
db = (ImageList.from_folder(path)
                   .split_none()
                   .label_from_folder()
                   .transform(get_transforms(), size=224)
                   .databunch())

In [ ]:
learn_cln = cnn_learner(db,models.resnet34,metrics=error_rate)

learn_cln.load("stage-2");

In [ ]:
ds,idxs = DatasetFormatter().from_toplosses(learn_cln)

In [ ]:
ImageCleaner(ds,idxs,path)

In [ ]:
ds,idxs= DatasetFormatter().from_similars(learn_cln)

In [ ]:
ImageCleaner(ds,idxs,path,duplicates=True)

In [ ]:
learn.export()

In [ ]:
defaults.device = torch.device("cpu")

In [ ]:
img = open_image("...")
img

In [ ]:
learn = load_learner(path)

In [ ]:
pred_class,pred_idx,outputs = learn.predict(img)
print(pred_class)